In [1]:
import os
import numpy as np

from sklearn.neighbors import BallTree
from sklearn.preprocessing import normalize

from scipy.spatial import distance

import pickle

from tqdm import tqdm

In [2]:
ddir = "/Users/rob/ML_Data/x86_vectors/lstm/"

ndims = 1000

In [3]:
def load_matrix(ndims):
    labels = []
    vec_buff = []
    base_names = []

    embeddings_dir = ddir + str(ndims) + "-raw/"
    for f in os.listdir(embeddings_dir):
        if f != "complete.txt":
            base_names.append(f.split(".")[0])
    base_names = list(set(base_names))

    for f in tqdm(base_names):
        try:
            vec_buff.append(np.load(embeddings_dir+f+".npy"))
            with open(embeddings_dir+f+".labels", "r") as label_file:
                for line in label_file:
                    labels.append(f+":"+line.strip())
        except:
            continue
                
    data_matrix = np.empty((len(labels), ndims))

    counter = 0

    for v in tqdm(vec_buff):
        data_matrix[counter:counter+v.shape[0],:] = v
        counter = counter + v.shape[0]
    
    return data_matrix, labels

In [4]:
def flatten(l, ltypes=(list, tuple)):
    ltype = type(l)
    l = list(l)
    i = 0
    while i < len(l):
        while isinstance(l[i], ltypes):
            if not l[i]:
                l.pop(i)
                i -= 1
                break
            else:
                l[i:i + 1] = l[i]
        i += 1
    return ltype(l)

In [5]:
data_matrix, labels = load_matrix(ndims)

In [15]:
c = 0
for d in data_matrix:
    for x in d:
        if np.isnan(x):
            c = c + 1
print c

139


In [10]:

data_matrix = normalize(data_matrix)
print data_matrix.shape
print data_matrix.nbytes

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
#tree = BallTree(data_matrix,leaf_size=100, metric=distance.cosine)
tree = BallTree(data_matrix,leaf_size=100, metric="euclidean")

In [ ]:
with open("/Users/rob/models/balltree-"+str(ndims)+".pickle","wb") as picklefile:
    pickle.dump(tree,picklefile)

In [ ]:
dist, ind = tree.query(data_matrix[10].reshape(1, -1), k=10)

In [ ]:
print ind
print dist

In [ ]:
print np.ndarray.flatten(ind)

In [ ]:
print labels[10]
for i in np.ndarray.flatten(ind):
    print labels[i]

In [ ]:
funcs_over_20 = []
indexes = []

with open("/Users/rob/data/10000_funcs_over_100_bytes.txt", "r") as func_file:
    for line in func_file:
        funcs_over_20.append(line.strip())

print "Loaded"
print len(funcs_over_20)
        
for func in funcs_over_20:
    if len(indexes) == 10000:
        break
    try:
        indexes.append(labels.index(func))
    except:
        continue
    
print len(indexes)

In [ ]:
print labels[0]
print funcs_over_20[0]

In [ ]:
dist_file = open("/Users/rob/data/neighbors-10k-lstm-"+str(ndims)+"-raw.txt", "w")
for i in tqdm(indexes):
    dist, ind = tree.query(data_matrix[i].reshape(1, -1), k=10)
    ind = np.ndarray.flatten(ind)
    dist = np.ndarray.flatten(dist)
    dist_file.write(labels[i] + ";")
    for index in xrange(10):
        dist_file.write(labels[ind[index]]+","+str(dist[index])+";")
    dist_file.write("\n")

dist_file.close()
    